In [1]:
import pandas as pd
# tables = pd.read_html('data/Data Grid.html')
# table = tables[0]
# table.columns = table.columns.droplevel()
# table.columns
# table.columns = ['Seed', 'Team Name', 'Round', 'Team', 'Pick', 'Odds', 'Public Pick%', 'Difference', 'Odds', 'Team', 'Added Risk', 'Impact']
# table = table[table['Seed'] != 'Seed']
# to_save = table[['Seed', 'Team Name', 'Round', 'Pick', 'Odds', 'Public Pick%']]
# to_save.to_csv('data/clean_data_for_parse.csv', index=False)
# regions = pd.read_excel('./data/March_Madness_2024_Silver_Bulletin_03_19_2024.xlsx')
# regions = regions[['team_seed', 'team_name', 'team_region']]
# regions.to_csv('data/regions.csv', index=False)

C:\Users\dmalo\AppData\Local\Temp\ipykernel_24308\3081817618.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
from fuzzywuzzy import process
import random

c:\Users\dmalo\anaconda3\envs\iball12\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
regions_df = pd.read_csv('data/regions.csv')
odds = pd.read_csv('data/clean_data_for_parse.csv')
regions_names = list(set(regions_df['team_name'].tolist()))
odds_names = list(set(odds['Team Name'].tolist()))
odds_to_regions = {}
for odds_name in odds_names:
    proc_out = process.extractOne(odds_name, regions_names)
    matched_name = proc_out[0]
    chance = proc_out[1]
    if chance > 70:
        if odds_name == 'Wash State':
            odds_to_regions[odds_name] = 'Washington State'
        elif odds_name == 'N Carolina':
            odds_to_regions[odds_name] = 'North Carolina'
        elif odds_name == 'NC State':
            odds_to_regions[odds_name] = 'North Carolina State'
        elif odds_name == 'S Dakota St':
            odds_to_regions[odds_name] = 'South Dakota State'
        elif odds_name == 'Colorado St':
            odds_to_regions[odds_name] = 'Colorado State'
        elif odds_name == 'S Carolina':
            odds_to_regions[odds_name] = 'South Carolina'
        elif odds_name == 'W Kentucky':
            odds_to_regions[odds_name] = 'Western Kentucky'
        elif odds_name == 'Miss State':
            odds_to_regions[odds_name] = 'Mississippi State'
        else:
            odds_to_regions[odds_name] = matched_name
    else:
        print(f'Failed to match {odds_name}, best match was {matched_name} with {chance}% chance.')
    
odds_to_regions["Fla Atlantic"] = "FAU"
odds_to_regions["TX Christian"] = "TCU"

odds['Team Name'] = odds['Team Name'].apply(lambda x: odds_to_regions[x] if x in odds_to_regions else x)
merged = odds.merge(regions_df, left_on='Team Name', right_on='team_name', how='left')
merged = merged.drop(columns=['team_name', 'Odds.1', 'team_seed'])
merged['Odds'] = merged['Odds'].apply(lambda x: x.replace('%', ''))
merged['Public Pick%'] = merged['Public Pick%'].apply(lambda x: x.replace('%', ''))
merged['Odds'] = pd.to_numeric(merged['Odds'])
merged['Public Pick%'] = pd.to_numeric(merged['Public Pick%'])
merged.to_csv('data/final_data.csv', index=False)

Failed to match TX Christian, best match was Charleston with 55% chance.
Failed to match Fla Atlantic, best match was TCU with 60% chance.


In [4]:
game_order = pd.read_csv('data/regions.csv')
order = game_order['team_name'].tolist()


In [5]:
# spec = [('name', string), ('seed', int32), ('region', string), ('rounds_odds', float32[:]), ('rounds_public_pick', float32[:])]
# @jitclass(spec)
class Team(object):
    def __init__(self, name, seed, region):
        self.name = name
        self.seed = seed
        self.region = region
        self.rounds_odds = []
        self.rounds_public_pick = []
        
    def __str__(self) -> str:
        return f'{self.name} ({self.seed})'


In [6]:
def sample_round_winner(first_team, second_team, round):
    my_odds = first_team.rounds_odds[round]
    other_odds = second_team.rounds_odds[round]
    total_odds = my_odds + other_odds
    my_chance = my_odds / total_odds
    random_number = random.random()
    if random_number < my_chance:
        return first_team
    else:
        return second_team
    
def sample_public_pick(first_team, second_team, round):
    first_pp = first_team.rounds_public_pick[round]
    second_pp = second_team.rounds_public_pick[round]
    total_pp = first_pp + second_pp
    first_chance = first_pp / total_pp
    random_number = random.random()
    if random_number < first_chance:
        return first_team
    else:
        return second_team

In [7]:
merged

,Seed,Team Name,Round,Pick,Odds,Public Pick%,team_region
0,1,Connecticut,7,To Win Tournament,22.8,30.4,East
1,1,Purdue,7,To Win Tournament,10.5,10.7,Midwest
2,1,Houston,7,To Win Tournament,11.9,13.0,South
3,2,Arizona,7,To Win Tournament,7.5,5.2,West
4,2,Tennessee,7,To Win Tournament,5.4,4.1,Midwest
...,...,...,...,...,...,...,...
379,15,Saint Peter's,2,To Make Round of 32,3.5,4.2,Midwest
380,16,Wagner,2,To Make Round of 32,0.7,1.5,West
381,16,Grambling State,2,To Make Round of 32,0.8,1.8,Midwest
382,16,Stetson,2,To Make Round of 32,0.5,1.5,East


In [8]:
teams = []
for name in order:
    data = merged[merged['Team Name'] == name]
    
    team = Team(name, data['Seed'].tolist()[0], data['team_region'].tolist()[0])
    for j in range(6):
        i = 5-j
        team.rounds_odds.append(data['Odds'].values[i])
        team.rounds_public_pick.append(data['Public Pick%'].values[i])
        
    teams.append(team)

In [9]:
def sample_tournament_outcome(teams):
    r32_winners = []
    r16_winners = []
    r8_winners = []
    r4_winners = []
    r2_winners = []
    champion = None


    for i in range(0, 64, 2):
        winner = sample_round_winner(teams[i], teams[i+1], 0)
        r32_winners.append(winner)

    for i in range(0, 32, 2):
        winner = sample_round_winner(r32_winners[i], r32_winners[i+1], 1)
        r16_winners.append(winner)
        

    for i in range(0, 16, 2):
        winner = sample_round_winner(r16_winners[i], r16_winners[i+1], 2)
        r8_winners.append(winner)


    for i in range(0, 8, 2):
        winner = sample_round_winner(r8_winners[i], r8_winners[i+1], 3)
        r4_winners.append(winner)

    for i in range(0, 4, 2):
        winner = sample_round_winner(r4_winners[i], r4_winners[i+1], 4)
        r2_winners.append(winner)

    for i in range(0, 2, 2):
        winner = sample_round_winner(r2_winners[i], r2_winners[i+1], 5)
        champion = winner
        
    tournament_result = [r32_winners, r16_winners, r8_winners, r4_winners, r2_winners, [champion]]
    tournament_result_names = [[team.name for team in round] for round in tournament_result]
    return tournament_result_names


In [10]:
outcome_test = sample_tournament_outcome(teams)
outcome_test


[['Connecticut',
  'Northwestern',
  'UAB',
  'Auburn',
  'BYU',
  'Illinois',
  'Drake',
  'Iowa State',
  'North Carolina',
  'Michigan State',
  "Saint Mary's College",
  'Alabama',
  'New Mexico',
  'Baylor',
  'Nevada',
  'Arizona',
  'Houston',
  'Nebraska',
  'Wisconsin',
  'Vermont',
  'Texas Tech',
  'Kentucky',
  'Colorado',
  'Western Kentucky',
  'Purdue',
  'TCU',
  'Gonzaga',
  'Kansas',
  'South Carolina',
  'Creighton',
  'Colorado State',
  'Tennessee'],
 ['Connecticut',
  'Auburn',
  'Illinois',
  'Iowa State',
  'North Carolina',
  'Alabama',
  'Baylor',
  'Arizona',
  'Nebraska',
  'Vermont',
  'Kentucky',
  'Colorado',
  'Purdue',
  'Gonzaga',
  'Creighton',
  'Tennessee'],
 ['Auburn',
  'Illinois',
  'North Carolina',
  'Baylor',
  'Nebraska',
  'Kentucky',
  'Purdue',
  'Creighton'],
 ['Auburn', 'Baylor', 'Kentucky', 'Creighton'],
 ['Auburn', 'Creighton'],
 ['Auburn']]

In [11]:
class Bracket(object):
    def __init__(self, teams):
        self.r32 = []
        self.r16 = []
        self.r8 = []
        self.r4 = []
        self.r2 = []
        self.champion = None
        
        for i in range(0, 64, 2):
            winner = sample_public_pick(teams[i], teams[i+1], 0)
            self.r32.append(winner)
        
        for i in range(0, 32, 2):
            winner = sample_public_pick(self.r32[i], self.r32[i+1], 1)
            self.r16.append(winner)
            
        for i in range(0, 16, 2):
            winner = sample_public_pick(self.r16[i], self.r16[i+1], 2)
            self.r8.append(winner)
            
        for i in range(0, 8, 2):
            winner = sample_public_pick(self.r8[i], self.r8[i+1], 3)
            self.r4.append(winner)
            
        for i in range(0, 4, 2):
            winner = sample_public_pick(self.r4[i], self.r4[i+1], 4)
            self.r2.append(winner)
            
        for i in range(0, 2, 2):
            winner = sample_public_pick(self.r2[i], self.r2[i+1], 5)
            self.champion = winner
            
        bracket_pick_names = [[team.name for team in round] for round in [self.r32, self.r16, self.r8, self.r4, self.r2, [self.champion]]]
        self.bracket_pick_names = bracket_pick_names
        
    def __eq__(self, __value: object) -> bool:
        sizes = [32, 16, 8, 4, 2, 1]
        for i in range(6):
            for j in range(sizes[i]):
                if self.bracket_pick_names[i][j] != __value.bracket_pick_names[i][j]:
                    return False
            
        return True
        
        
    def score_bracket(self, tournament_result):
        score = 0
        for i in range(6):
            this_picks = self.bracket_pick_names[i]
            actual = tournament_result[i]
            round_score = 2**i
            score += len(set(this_picks).intersection(set(actual))) * round_score
        return score
        
    

In [12]:
def sample_pool_outcome(pool_size, teams, payouts, num_tournaments=1000):
    pool = []
    for i in range(pool_size):
        pool.append(Bracket(teams))
        
    winnings_tally = [0 for i in range(pool_size)]
    
    for i in range(num_tournaments):
        tournament_result = sample_tournament_outcome(teams)
        outcomes = []
        for i in range(pool_size):
            bracket_score = pool[i].score_bracket(tournament_result)
            outcomes.append((i, bracket_score))
            
        
        outcomes = sorted(outcomes, key=lambda x: x[1], reverse=True)
        scores = [x[1] for x in outcomes]
        best_score = scores[0]
        best_indices = [x[0] for x in outcomes if x[1] == best_score]
        without_best = [x for x in outcomes if x[1] != best_score]
        second_best_score = without_best[0][1]
        second_best_indices = [x[0] for x in without_best if x[1] == second_best_score]
        
        for best_index in best_indices:
            winnings_tally[best_index] += 1 * payouts[0]
            
        for second_best_index in second_best_indices:
            winnings_tally[second_best_index] += 1 * payouts[1]        
                
    average_winnings = [(pool[i], (winnings_tally[i] / num_tournaments)) for i in range(pool_size)]
    average_winnings = sorted(average_winnings, key=lambda x: x[1], reverse=True)
    return average_winnings

In [13]:
import tqdm

In [14]:
def get_best_brackets(pool_size, teams, payouts, num_tournaments=1000, num_pool_sims=10):
    best_brackets = []
    best_scores = []
    for i in tqdm.tqdm(range(num_pool_sims)):
        outcome = sample_pool_outcome(pool_size, teams, payouts, num_tournaments)
        best = outcome[0]
        bb = best[0]
        bs = best[1]
        if bb not in best_brackets:
            best_brackets.append(bb)
            best_scores.append(bs)
        elif bs > best_scores[best_brackets.index(bb)]:
            best_scores[best_brackets.index(bb)] = bs
        
        
    combined = [(brack, score) for brack, score in zip(best_brackets, best_scores)]
    
    return combined


In [15]:
best_from_sims = get_best_brackets(220, teams, [300, 0], num_tournaments=1000, num_pool_sims=10)

100%|██████████| 10/10 [00:17<00:00,  1.73s/it]


In [16]:
best_from_sims = sorted(best_from_sims, key=lambda x: x[1], reverse=True)
best_from_sims[0][0].bracket_pick_names

[['Connecticut',
  'FAU',
  'San Diego State',
  'Auburn',
  'BYU',
  'Illinois',
  'Drake',
  'Iowa State',
  'Wagner',
  'Michigan State',
  "Saint Mary's College",
  'Alabama',
  'New Mexico',
  'Baylor',
  'Nevada',
  'Arizona',
  'Houston',
  'Texas A&M',
  'James Madison',
  'Duke',
  'North Carolina State',
  'Kentucky',
  'Florida',
  'Marquette',
  'Purdue',
  'TCU',
  'Gonzaga',
  'Kansas',
  'South Carolina',
  'Creighton',
  'Texas',
  'Tennessee'],
 ['Connecticut',
  'Auburn',
  'BYU',
  'Iowa State',
  'Michigan State',
  'Alabama',
  'New Mexico',
  'Arizona',
  'Houston',
  'Duke',
  'Kentucky',
  'Florida',
  'Purdue',
  'Gonzaga',
  'Creighton',
  'Tennessee'],
 ['Connecticut',
  'Iowa State',
  'Alabama',
  'Arizona',
  'Duke',
  'Kentucky',
  'Purdue',
  'Tennessee'],
 ['Connecticut', 'Arizona', 'Duke', 'Tennessee'],
 ['Connecticut', 'Tennessee'],
 ['Tennessee']]

In [17]:
len(best_from_sims)


10

In [18]:
import numpy as np
from numpy import int8, float32
from numba import jit
from numba import prange

In [19]:
teams_odds = np.zeros((6, 64))
public_odds = np.zeros((6, 64))
teamid_to_name = {}
for i in range(len(order)):
    name = order[i]
    team_id = i
    teamid_to_name[team_id] = name
    data = merged[merged['Team Name'] == name]
    
    #print(data)
    for j in range(6):
        z = 5-j
        #print(data['Public Pick%'].values[z])
        teams_odds[j][i] = data['Odds'].values[z]
        public_odds[j][i] = data['Public Pick%'].values[z]
        
    

In [ ]:
@jit(nopython=True)
def pick_one_bracket(public_odds):

    r64_matchups = public_odds[0].reshape(32, 2)
    r32_raw = public_odds[1]

    r32 = np.zeros(32)
    r32_teams = np.zeros(32, dtype=int8)

    r64_randoms = np.random.random(32)
    r64_odds= r64_matchups[:,0]/r64_matchups.sum(axis=1)
    r64_winners = r64_randoms > r64_odds
    r64_winners = r64_winners.astype(int8)

    for i in range(32):
        r32_teams[i] = i*2 + r64_winners[i]
        r32[i] = r32_raw[i*2 + r64_winners[i]]
        
    r32_matchups = r32.reshape(16, 2)

    r16_raw = public_odds[2]
    r16 = np.zeros(16)
    r16_teams = np.zeros(16, dtype=int8)
    r32_randoms = np.random.random(16)
    r32_odds = r32_matchups[:,0]/r32_matchups.sum(axis=1)
    r32_winners = r32_randoms > r32_odds
    r32_winners = r32_winners.astype(int8)

    for i in range(16):
        r16_teams[i] = r32_teams[i*2 + r32_winners[i]]
        r16[i] = r16_raw[r32_teams[i*2 + r32_winners[i]]]


    r16_matchups = r16.reshape(8, 2)
    r8_raw = public_odds[3]
    r8 = np.zeros(8)
    r8_teams = np.zeros(8, dtype=int8)
    r16_randoms = np.random.random(8)
    r16_odds = r16_matchups[:,0]/r16_matchups.sum(axis=1)
    r16_winners = r16_randoms > r16_odds
    r16_winners = r16_winners.astype(int8)

    for i in range(8):
        r8_teams[i] = r16_teams[i*2 + r16_winners[i]]
        r8[i] = r8_raw[r16_teams[i*2 + r16_winners[i]]]
        
    r8_matchups = r8.reshape(4, 2)
    r4_raw = public_odds[4]
    r4 = np.zeros(4)
    r4_teams = np.zeros(4, dtype=int8)
    r8_randoms = np.random.random(4)
    r8_odds = r8_matchups[:,0]/r8_matchups.sum(axis=1)
    r8_winners = r8_randoms > r8_odds
    r8_winners = r8_winners.astype(int8)

    for i in range(4):
        r4_teams[i] = r8_teams[i*2 + r8_winners[i]]
        r4[i] = r4_raw[r8_teams[i*2 + r8_winners[i]]]
        
    r4_matchups = r4.reshape(2, 2)
    r2_raw = public_odds[5]
    r2 = np.zeros(2)
    r2_teams = np.zeros(2, dtype=int8)
    r4_randoms = np.random.random(2)
    r4_odds = r4_matchups[:,0]/r4_matchups.sum(axis=1)
    r4_winners = r4_randoms > r4_odds
    r4_winners = r4_winners.astype(int8)

    for i in range(2):
        r2_teams[i] = r4_teams[i*2 + r4_winners[i]]
        r2[i] = r2_raw[r4_teams[i*2 + r4_winners[i]]]
        

    r2_matchups = r2.reshape(1, 2)
    r2_randoms = np.random.random(1)
    r2_odds = r2_matchups[:,0]/r2_matchups.sum(axis=1)
    r2_winners = r2_randoms > r2_odds
    r2_winners = r2_winners.astype(int8)

    final_winner = r2_teams[r2_winners[0]]
    fw = np.zeros(1)
    fw[0] = final_winner

    #round_winners = np.array([r32_teams, np.pad(r16_teams, (0, 16), 'constant', constant_values=-1), np.pad(r8_teams, (0, 24), 'constant', constant_values=-1), np.pad(r4_teams, (0, 28), 'constant', constant_values=-1), np.pad(r2_teams, (0, 30), 'constant', constant_values=-1), np.pad([final_winner], (0, 31), 'constant', constant_values=-1)])
    #round_winners = np.concatenate((r32_teams, r16_teams, r8_teams, r4_teams, r2_teams, final_winner), axis=None)
    rw = np.zeros(63, dtype=int8)
    rw[:32] = r32_teams
    rw[32:48] = r16_teams
    rw[48:56] = r8_teams
    rw[56:60] = r4_teams
    rw[60:62] = r2_teams
    rw[62] = final_winner
    return rw
    
    

pick_one_bracket(public_odds)
@jit(nopython=True)
def simulate_tournament_results(teams_odds):

    r64_matchups = teams_odds[0].reshape(32, 2)
    r32_raw = teams_odds[1]
    
    r32 = np.zeros(32)
    r32_teams = np.zeros(32, dtype=int8)

    r64_randoms = np.random.random(32)
    r64_odds= r64_matchups[:,0]/r64_matchups.sum(axis=1)
    r64_winners = r64_randoms > r64_odds
    r64_winners = r64_winners.astype(int8)

    for i in range(32):
        r32_teams[i] = i*2 + r64_winners[i]
        r32[i] = r32_raw[i*2 + r64_winners[i]]
        
    r32_matchups = r32.reshape(16, 2)

    r16_raw = teams_odds[2]
    r16 = np.zeros(16)
    r16_teams = np.zeros(16, dtype=int8)
    r32_randoms = np.random.random(16)
    r32_odds = r32_matchups[:,0]/r32_matchups.sum(axis=1)
    r32_winners = r32_randoms > r32_odds
    r32_winners = r32_winners.astype(int8)

    for i in range(16):
        r16_teams[i] = r32_teams[i*2 + r32_winners[i]]
        r16[i] = r16_raw[r32_teams[i*2 + r32_winners[i]]]


    r16_matchups = r16.reshape(8, 2)
    r8_raw = teams_odds[3]
    r8 = np.zeros(8)
    r8_teams = np.zeros(8, dtype=int8)
    r16_randoms = np.random.random(8)
    r16_odds = r16_matchups[:,0]/r16_matchups.sum(axis=1)
    r16_winners = r16_randoms > r16_odds
    r16_winners = r16_winners.astype(int8)

    for i in range(8):
        r8_teams[i] = r16_teams[i*2 + r16_winners[i]]
        r8[i] = r8_raw[r16_teams[i*2 + r16_winners[i]]]
        
    r8_matchups = r8.reshape(4, 2)
    r4_raw = teams_odds[4]
    r4 = np.zeros(4)
    r4_teams = np.zeros(4, dtype=int8)
    r8_randoms = np.random.random(4)
    r8_odds = r8_matchups[:,0]/r8_matchups.sum(axis=1)
    r8_winners = r8_randoms > r8_odds
    r8_winners = r8_winners.astype(int8)

    for i in range(4):
        r4_teams[i] = r8_teams[i*2 + r8_winners[i]]
        r4[i] = r4_raw[r8_teams[i*2 + r8_winners[i]]]
        
    r4_matchups = r4.reshape(2, 2)
    r2_raw = teams_odds[5]
    r2 = np.zeros(2)
    r2_teams = np.zeros(2, dtype=int8)
    r4_randoms = np.random.random(2)
    r4_odds = r4_matchups[:,0]/r4_matchups.sum(axis=1)
    r4_winners = r4_randoms > r4_odds
    r4_winners = r4_winners.astype(int8)

    for i in range(2):
        r2_teams[i] = r4_teams[i*2 + r4_winners[i]]
        r2[i] = r2_raw[r4_teams[i*2 + r4_winners[i]]]
        

    r2_matchups = r2.reshape(1, 2)
    r2_randoms = np.random.random(1)
    r2_odds = r2_matchups[:,0]/r2_matchups.sum(axis=1)
    r2_winners = r2_randoms > r2_odds
    r2_winners = r2_winners.astype(int8)

    final_winner = r2_teams[r2_winners[0]]


    rw = np.zeros(63, dtype=int8)
    rw[:32] = r32_teams
    rw[32:48] = r16_teams
    rw[48:56] = r8_teams
    rw[56:60] = r4_teams
    rw[60:62] = r2_teams
    rw[62] = final_winner
    
    return rw
    
    

simulate_tournament_results(teams_odds)
@jit(nopython=True)
def score_bracket(bracket, tournament_result):
    score = 0
    
    for i in range(63):
        if bracket[i] == tournament_result[i]:
            if i < 32:
                score += 1
            elif i < 48:
                score += 2
            elif i < 56:
                score += 4
            elif i < 60:
                score += 8
            elif i < 62:
                score += 16
            else:
                score += 32
        #print(score)
    
    return score
@jit(nopython=True, parallel=True)
def simulate_pool(teams_odds, public_odds, pool_size, payouts, num_tournaments=1000):
    pool = np.zeros((pool_size, 63), dtype=int8)
    for i in prange(pool_size):
        pool[i] = pick_one_bracket(public_odds)
        
    #winnings_tally = np.zeros(pool_size, dtype=float32)
    all_winnings = np.zeros((num_tournaments, pool_size), dtype=float32)
    for i in prange(num_tournaments):
        tournament_result = simulate_tournament_results(teams_odds)
        scores_list = np.zeros(pool_size, dtype=float32)
        for z in prange(pool_size):
            bracket_score = score_bracket(pool[z], tournament_result)
            scores_list[z] = bracket_score
                
            
        
        unique_scores = np.unique(scores_list)
        unique_scores.sort()
        unique_scores = unique_scores[::-1]
        
        for p in range(len(payouts)):
            payout = payouts[p]
            count = (scores_list == unique_scores[p]).sum()
            is_max = np.argwhere(scores_list == unique_scores[p])
            for j in range(count):
                all_winnings[i][is_max[j]] = payout/count
       
                
    summed_winnings = np.sum(all_winnings, axis=0)
    average_winnings = summed_winnings / num_tournaments
    
    order = np.argsort(average_winnings)[::-1]
    
    #average_winnings = [(pool[i], average_winnings[i]) for i in range(pool_size)]
    #average_winnings = sorted(average_winnings, key=lambda x: x[1], reverse=True)
    
    best_winnings = average_winnings[order[0]]
    best_pool = pool[order[0]]
    
    out = np.zeros(64, dtype=float32)
    out[:63] = best_pool
    out[63] = best_winnings
    return out
simulate_pool(teams_odds, public_odds, 220, [400, 20], num_tournaments=1000)
@jit(nopython=True, parallel=True)
def get_best_brackets(teams_odds, public_odds, pool_size, payouts, num_tournaments=1000, num_pool_sims=10):
    best_brackets = np.zeros((num_pool_sims, 63), dtype=int8)
    best_scores = np.zeros(num_pool_sims, dtype=float32)
    for i in prange(num_pool_sims):
        outcome = simulate_pool(teams_odds, public_odds, pool_size, payouts, num_tournaments)
        bb = outcome[0:63].astype(int8)
        bs = outcome[63]
        best_brackets[i] = bb
        best_scores[i] = bs
        #print(bb)
        
        
    # if len(best_brackets) == 0:
    #     best_brackets.append(bb)
    #     best_scores.append(bs)
    # else:
    #     in_best = False
    #     for i in range(len(best_brackets)):
    #         if (bb == best_brackets[i]).all():
    #             in_best = True
    #             if bs > best_scores[i]:
    #                 best_scores[i] = bs
    #             break
    #     if not in_best:
    #         best_brackets.append(bb)
    #         best_scores.append(bs)
        
        
    combined = [(brack, score) for brack, score in zip(best_brackets, best_scores)]
    
    return combined

In [37]:
test= get_best_brackets(teams_odds, public_odds, 22, [400, 40], num_tournaments=10000, num_pool_sims=100)
test = sorted(test, key=lambda x: x[1], reverse=True)
print(test[0][1], [teamid_to_name[x] for x in test[0][0]])

78.28165435791016 ['Connecticut', 'Northwestern', 'San Diego State', 'Auburn', 'Duquesne', 'Illinois', 'Washington State', 'Iowa State', 'North Carolina', 'Mississippi State', 'Grand Canyon', 'Alabama', 'Clemson', 'Baylor', 'Dayton', 'Arizona', 'Houston', 'Texas A&M', 'Wisconsin', 'Duke', 'Texas Tech', 'Kentucky', 'Florida', 'Marquette', 'Purdue', 'TCU', 'Gonzaga', 'Kansas', 'Oregon', 'Creighton', 'Colorado State', 'Tennessee', 'Connecticut', 'Auburn', 'Illinois', 'Iowa State', 'North Carolina', 'Alabama', 'Baylor', 'Arizona', 'Houston', 'Duke', 'Kentucky', 'Marquette', 'Purdue', 'Gonzaga', 'Creighton', 'Tennessee', 'Connecticut', 'Iowa State', 'North Carolina', 'Arizona', 'Houston', 'Marquette', 'Purdue', 'Creighton', 'Connecticut', 'Arizona', 'Houston', 'Purdue', 'Arizona', 'Purdue', 'Purdue']


In [38]:
%%time
get_best_brackets(teams_odds, public_odds, 40, [800, 100, 20], num_tournaments=10000, num_pool_sims=1000)


CPU times: total: 6min 37s
Wall time: 29.6 s


[(array([ 0,  2,  4,  6,  8, 10, 13, 14, 16, 18, 21, 22, 25, 26, 28, 30, 32,
         35, 36, 38, 41, 42, 45, 46, 48, 50, 52, 54, 57, 58, 60, 62,  0,  6,
          8, 14, 16, 21, 26, 30, 32, 38, 41, 46, 48, 54, 58, 62,  0,  8, 16,
         30, 32, 46, 48, 62,  0, 30, 32, 48,  0, 48, 48], dtype=int8),
  57.73633575439453),
 (array([ 0,  3,  4,  6,  8, 10, 12, 14, 16, 19, 20, 22, 24, 26, 29, 30, 32,
         35, 36, 38, 40, 42, 45, 46, 48, 51, 52, 54, 56, 58, 60, 62,  0,  4,
          8, 14, 16, 22, 26, 30, 32, 38, 42, 46, 48, 52, 58, 62,  0, 14, 16,
         30, 38, 42, 48, 62,  0, 16, 38, 48,  0, 48,  0], dtype=int8),
  46.79866027832031),
 (array([ 0,  2,  4,  6,  8, 10, 13, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32,
         34, 37, 38, 41, 42, 45, 46, 48, 51, 52, 54, 57, 58, 60, 62,  0,  6,
          8, 14, 16, 22, 26, 30, 32, 38, 41, 46, 48, 52, 58, 62,  0, 14, 22,
         30, 32, 46, 48, 62, 14, 30, 46, 48, 30, 48, 48], dtype=int8),
  71.76483154296875),
 (array([ 0,  3,  5,  6,  9,

In [39]:
out = get_best_brackets(teams_odds, public_odds, 250, [400, 20], num_tournaments=10000, num_pool_sims=10000)
out = sorted(out, key=lambda x: x[1], reverse=True)
print(out[0][1], [teamid_to_name[x] for x in out[0][0]])

24.704334259033203 ['Connecticut', 'FAU', 'San Diego State', 'Auburn', 'BYU', 'Illinois', 'Washington State', 'Iowa State', 'North Carolina', 'Mississippi State', 'Grand Canyon', 'Alabama', 'Clemson', 'Baylor', 'Dayton', 'Arizona', 'Houston', 'Texas A&M', 'Wisconsin', 'Duke', 'Texas Tech', 'Kentucky', 'Florida', 'Marquette', 'Purdue', 'Utah State', 'Gonzaga', 'Samford', 'Oregon', 'Creighton', 'Colorado State', 'Tennessee', 'Connecticut', 'Auburn', 'Illinois', 'Iowa State', 'North Carolina', 'Alabama', 'Baylor', 'Arizona', 'Houston', 'Duke', 'Kentucky', 'Marquette', 'Purdue', 'Gonzaga', 'Oregon', 'Tennessee', 'Connecticut', 'Iowa State', 'North Carolina', 'Arizona', 'Houston', 'Marquette', 'Purdue', 'Tennessee', 'Iowa State', 'Arizona', 'Houston', 'Purdue', 'Arizona', 'Purdue', 'Arizona']


In [274]:
output = get_best_brackets(teams_odds, public_odds, 220, [400, 20], num_tournaments=10000, num_pool_sims=10000)
output = sorted(output, key=lambda x: x[1], reverse=True)
print(output[0][1], [teamid_to_name.get(i) for i in output[0][0]])

[(array([ 0.,  2.,  4.,  6.,  8., 10., 12., 14., 16., 18., 20., 22., 25.,
         26., 28., 30., 32., 34., 36., 38., 41., 42., 45., 46., 48., 51.,
         52., 55., 56., 58., 60., 62.,  0.,  6., 10., 14., 16., 22., 26.,
         30., 32., 38., 42., 46., 48., 52., 58., 62.,  0., 14., 16., 30.,
         32., 42., 48., 62.,  0., 30., 32., 48., 30., 48., 30.]),
  7.2),
 (array([ 0.,  3.,  5.,  6.,  8., 10., 12., 14., 16., 18., 20., 22., 25.,
         26., 28., 30., 32., 35., 37., 38., 41., 42., 44., 46., 48., 50.,
         52., 54., 57., 58., 61., 62.,  3.,  6., 10., 14., 18., 22., 26.,
         28., 32., 38., 42., 46., 48., 54., 58., 62.,  6., 10., 22., 26.,
         32., 46., 48., 58.,  6., 26., 46., 48.,  6., 46.,  6.]),
  6.6),
 (array([ 0.,  2.,  5.,  6.,  9., 10., 13., 14., 16., 18., 20., 22., 24.,
         26., 29., 30., 32., 34., 36., 38., 41., 42., 44., 46., 48., 51.,
         52., 54., 56., 58., 60., 62.,  2.,  6., 10., 14., 16., 20., 24.,
         30., 32., 38., 42., 46., 48.,